<h1><center>Laboratorio 9: Optimización de modelos 💯</center></h1>

<center><strong>MDS7202: Laboratorio de Programación Científica para Ciencia de Datos - Primavera 2024</strong></center>

### **Cuerpo Docente:**

- Profesores: Ignacio Meza, Sebastián Tinoco
- Auxiliar: Eduardo Moya
- Ayudantes: Nicolás Ojeda, Melanie Peña, Valentina Rojas

### Equipo: SUPER IMPORTANTE - notebooks sin nombre no serán revisados

- Nombre de alumno 1: Lucas Carrasco
- Nombre de alumno 2: Nicolás Herrera


### **Link de repositorio de GitHub:** [Insertar Repositorio](https://github.com/vspartamo/MDS7202)

### Temas a tratar

- Predicción de demanda usando `xgboost`
- Búsqueda del modelo óptimo de clasificación usando `optuna`
- Uso de pipelines.

### Reglas:

- **Grupos de 2 personas**
- Cualquier duda fuera del horario de clases al foro. Mensajes al equipo docente serán respondidos por este medio.
- Prohibidas las copias.
- Pueden usar cualquer matrial del curso que estimen conveniente.
- Código que no se pueda ejecutar, no será revisado.

### Objetivos principales del laboratorio

- Optimizar modelos usando `optuna`
- Recurrir a técnicas de *prunning*
- Forzar el aprendizaje de relaciones entre variables mediante *constraints*
- Fijar un pipeline con un modelo base que luego se irá optimizando.

El laboratorio deberá ser desarrollado sin el uso indiscriminado de iteradores nativos de python (aka "for", "while"). La idea es que aprendan a exprimir al máximo las funciones optimizadas que nos entrega `pandas`, las cuales vale mencionar, son bastante más eficientes que los iteradores nativos sobre DataFrames.

# Importamos librerias útiles

In [ ]:
!pip install -qq xgboost optuna

# El emprendimiento de Fiu

Tras liderar de manera exitosa la implementación de un proyecto de ciencia de datos para caracterizar los datos generados en Santiago 2023, el misterioso corpóreo **Fiu** se anima y decide levantar su propio negocio de consultoría en machine learning. Tras varias e intensas negociaciones, Fiu logra encontrar su *primera chamba*: predecir la demanda (cantidad de venta) de una famosa productora de bebidas de calibre mundial. Al ver el gran potencial y talento que usted ha demostrado en el campo de la ciencia de datos, Fiu lo contrata como data scientist para que forme parte de su nuevo emprendimiento.

Para este laboratorio deben trabajar con los datos `sales.csv` subidos a u-cursos, el cual contiene una muestra de ventas de la empresa para diferentes productos en un determinado tiempo.

Para comenzar, cargue el dataset señalado y visualice a través de un `.head` los atributos que posee el dataset.

<i><p align="center">Fiu siendo felicitado por su excelente desempeño en el proyecto de caracterización de datos</p></i>
<p align="center">
  <img src="https://media-front.elmostrador.cl/2023/09/A_UNO_1506411_2440e.jpg">
</p>

In [133]:
import pandas as pd
from datetime import datetime

df = pd.read_csv("sales.csv")

df.head()

,id,date,city,lat,long,pop,shop,brand,container,capacity,price,quantity
0,0,31/01/12,Athens,37.97945,23.71622,672130,shop_1,kinder-cola,glass,500ml,0.96,13280
1,1,31/01/12,Athens,37.97945,23.71622,672130,shop_1,kinder-cola,plastic,1.5lt,2.86,6727
2,2,31/01/12,Athens,37.97945,23.71622,672130,shop_1,kinder-cola,can,330ml,0.87,9848
3,3,31/01/12,Athens,37.97945,23.71622,672130,shop_1,adult-cola,glass,500ml,1.00,20050
4,4,31/01/12,Athens,37.97945,23.71622,672130,shop_1,adult-cola,can,330ml,0.39,25696


In [134]:
def convert_capacity_to_ml(capacity):
    if 'ml' in capacity:
        return float(capacity.replace('ml', ''))
    elif 'lt' in capacity:
        return float(capacity.replace('lt', '')) * 1000
    else:
        raise ValueError(f"Unknown capacity format: {capacity}")
df['capacity'] = df['capacity'].apply(convert_capacity_to_ml)

#convertir las columnas lat, long, pop, capacity, price y quantity como variables numericas
df['lat'] = pd.to_numeric(df['lat'], errors='coerce')
df['long'] = pd.to_numeric(df['long'], errors='coerce')
df['pop'] = pd.to_numeric(df['pop'], errors='coerce')
df['capacity'] = pd.to_numeric(df['capacity'], errors='coerce')
df['price'] = pd.to_numeric(df['price'], errors='coerce')
df['quantity'] = pd.to_numeric(df['quantity'], errors='coerce')
#convertir las columnas city, shop, brand y container a categoricas
df['city'] = df['city'].astype('category')
df['shop'] = df['shop'].astype('category')
df['brand'] = df['brand'].astype('category')
df['container'] = df['container'].astype('category')
#convertir la columna date a datetime
df['date'] = pd.to_datetime(df['date'])
df.head()

,id,date,city,lat,long,pop,shop,brand,container,capacity,price,quantity
0,0,2012-01-31,Athens,37.97945,23.71622,672130,shop_1,kinder-cola,glass,500.0,0.96,13280
1,1,2012-01-31,Athens,37.97945,23.71622,672130,shop_1,kinder-cola,plastic,1500.0,2.86,6727
2,2,2012-01-31,Athens,37.97945,23.71622,672130,shop_1,kinder-cola,can,330.0,0.87,9848
3,3,2012-01-31,Athens,37.97945,23.71622,672130,shop_1,adult-cola,glass,500.0,1.00,20050
4,4,2012-01-31,Athens,37.97945,23.71622,672130,shop_1,adult-cola,can,330.0,0.39,25696


In [135]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7456 entries, 0 to 7455
Data columns (total 12 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   id         7456 non-null   int64         
 1   date       7456 non-null   datetime64[ns]
 2   city       7456 non-null   category      
 3   lat        7456 non-null   float64       
 4   long       7456 non-null   float64       
 5   pop        7456 non-null   int64         
 6   shop       7456 non-null   category      
 7   brand      7456 non-null   category      
 8   container  7456 non-null   category      
 9   capacity   7456 non-null   float64       
 10  price      7456 non-null   float64       
 11  quantity   7456 non-null   int64         
dtypes: category(4), datetime64[ns](1), float64(4), int64(3)
memory usage: 496.0 KB


## 1 Generando un Baseline (5 puntos)

<p align="center">
  <img src="https://media.tenor.com/O-lan6TkadUAAAAC/what-i-wnna-do-after-a-baseline.gif">
</p>

Antes de entrenar un algoritmo, usted recuerda los apuntes de su magíster en ciencia de datos y recuerda que debe seguir una serie de *buenas prácticas* para entrenar correcta y debidamente su modelo. Después de un par de vueltas, llega a las siguientes tareas:

1. Separe los datos en conjuntos de train (70%), validation (20%) y test (10%). Fije una semilla para controlar la aleatoriedad. [0.5 puntos]
2. Implemente un `FunctionTransformer` para extraer el día, mes y año de la variable `date`. Guarde estas variables en el formato categorical de pandas. [1 punto]
3. Implemente un `ColumnTransformer` para procesar de manera adecuada los datos numéricos y categóricos. Use `OneHotEncoder` para las variables categóricas. `Nota:` Utilice el método `.set_output(transform='pandas')` para obtener un DataFrame como salida del `ColumnTransformer` [1 punto]
4. Guarde los pasos anteriores en un `Pipeline`, dejando como último paso el regresor `DummyRegressor` para generar predicciones en base a promedios. [0.5 punto]
5. Entrene el pipeline anterior y reporte la métrica `mean_absolute_error` sobre los datos de validación. ¿Cómo se interpreta esta métrica para el contexto del negocio? [0.5 puntos]
6. Finalmente, vuelva a entrenar el `Pipeline` pero esta vez usando `XGBRegressor` como modelo **utilizando los parámetros por default**. ¿Cómo cambia el MAE al implementar este algoritmo? ¿Es mejor o peor que el `DummyRegressor`? [1 punto]
7. Guarde ambos modelos en un archivo .pkl (uno cada uno) [0.5 puntos]

In [136]:
from sklearn import set_config

from sklearn.model_selection import train_test_split

X = df.drop(columns=["quantity"])
y = df["quantity"]

X_train, X_val_test, y_train, y_val_test = train_test_split(X, y, test_size=0.3, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_val_test, y_val_test, test_size=0.33, random_state=42)
#mostrar las dimensiones de los conjuntos de datos
X_train.shape, X_val.shape, X_test.shape



((5219, 11), (1498, 11), (739, 11))

In [137]:
from sklearn.preprocessing import FunctionTransformer
def extract_date_features(df):
    df['date'] = pd.to_datetime(df['date'], format='%d/%m/%y')
    df['day'] = df['date'].dt.day.astype('category')
    df['month'] = df['date'].dt.month.astype('category')
    df['year'] = df['date'].dt.year.astype('category')
    #df.drop(columns=['date'], inplace=True)
    return df

date_transformer = FunctionTransformer(extract_date_features, validate=False)

In [138]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler

numerical_cols = ['lat', 'long', 'pop', 'capacity', 'price']
categorical_cols = ['city', 'shop', 'brand', 'container', 'day', 'month', 'year']

preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_cols),
        ('cat', OneHotEncoder(handle_unknown='ignore',sparse_output=False), categorical_cols)
    ]
).set_output(transform= 'pandas')

In [139]:
from sklearn.dummy import DummyRegressor
from sklearn.pipeline import Pipeline

pipeline_dummy = Pipeline([
    ('date_transformer', date_transformer),
    ('preprocessor', preprocessor),
    ('regressor', DummyRegressor(strategy='mean'))
])


In [140]:
from sklearn.metrics import mean_absolute_error

# Entrenar pipeline
pipeline_dummy.fit(X_train, y_train)

# Predicciones en el conjunto de validación
val_preds = pipeline_dummy.predict(X_val)

# Calcular MAE
mae_dummy = mean_absolute_error(y_val, val_preds)
print(f"MAE (DummyRegressor): {mae_dummy}")


MAE (DummyRegressor): 13308.134750658153


In [141]:
# predicciones en el conjunto de test
test_preds = pipeline_dummy.predict(X_test)
# calcular MAE
mae_dummy_test = mean_absolute_error(y_test, test_preds)
print(f"MAE (DummyRegressor) en test: {mae_dummy_test}")

MAE (DummyRegressor) en test: 13394.728221101155


In [142]:
from xgboost import XGBRegressor

# Define the pipeline with XGBRegressor
pipeline_xgb = Pipeline([
    ('date_transformer', date_transformer),
    ('preprocessor', preprocessor),
    ('regressor', XGBRegressor(random_state=3))
])

# Train the pipeline
pipeline_xgb.fit(X_train, y_train)

# Make predictions on the validation set
val_preds_xgb = pipeline_xgb.predict(X_val)

# Calculate MAE
mae_xgb = mean_absolute_error(y_val, val_preds_xgb)
print(f"MAE (XGBRegressor): {mae_xgb}")

MAE (XGBRegressor): 2420.2877598190817


In [143]:
# predictions in the test set
test_preds_xgb = pipeline_xgb.predict(test_df)
# calculate MAE
mae_xgb_test = mean_absolute_error(test_df['quantity'], test_preds_xgb)
print(f"MAE (XGBRegressor) in test: {mae_xgb_test}")

MAE (XGBRegressor) in test: 1545.2211458157137


In [144]:
import joblib

# Guardar los modelos
joblib.dump(pipeline_dummy, 'pipeline_dummy.pkl')
joblib.dump(pipeline_xgb, 'pipeline_xgb.pkl')

['pipeline_xgb.pkl']

## 2. Forzando relaciones entre parámetros con XGBoost (10 puntos)

<p align="center">
  <img src="https://64.media.tumblr.com/14cc45f9610a6ee341a45fd0d68f4dde/20d11b36022bca7b-bf/s640x960/67ab1db12ff73a530f649ac455c000945d99c0d6.gif">
</p>

Un colega aficionado a la economía le *sopla* que la demanda guarda una relación inversa con el precio del producto. Motivado para impresionar al querido corpóreo, se propone hacer uso de esta información para mejorar su modelo realizando las siguientes tareas:

1. Vuelva a entrenar el `Pipeline` con `XGBRegressor`, pero esta vez forzando una relación monótona negativa entre el precio y la cantidad. Para aplicar esta restricción apóyese en la siguiente <a href = https://xgboost.readthedocs.io/en/stable/tutorials/monotonic.html>documentación</a>. [6 puntos]

>Hint 1: Para implementar el constraint se le sugiere hacerlo especificando el nombre de la variable. De ser así, probablemente le sea útil **mantener el formato de pandas** antes del step de entrenamiento.

>Hint 2: Puede obtener el nombre de las columnas en el paso anterior al modelo regresor mediante el método `.get_feature_names_out()`

2. Luego, vuelva a reportar el `MAE` sobre el conjunto de validación. [1 puntos]

3. ¿Cómo cambia el error al incluir esta relación? ¿Tenía razón su amigo? [2 puntos]

4. Guarde su modelo en un archivo .pkl [1 punto]

In [145]:
feature_names = preprocessor.get_feature_names_out()
print(feature_names)

['num__lat' 'num__long' 'num__pop' 'num__capacity' 'num__price'
 'cat__city_Athens' 'cat__city_Irakleion' 'cat__city_Larisa'
 'cat__city_Patra' 'cat__city_Thessaloniki' 'cat__shop_shop_1'
 'cat__shop_shop_2' 'cat__shop_shop_3' 'cat__shop_shop_4'
 'cat__shop_shop_5' 'cat__shop_shop_6' 'cat__brand_adult-cola'
 'cat__brand_gazoza' 'cat__brand_kinder-cola' 'cat__brand_lemon-boost'
 'cat__brand_orange-power' 'cat__container_can' 'cat__container_glass'
 'cat__container_plastic' 'cat__day_28' 'cat__day_29' 'cat__day_30'
 'cat__day_31' 'cat__month_1' 'cat__month_2' 'cat__month_3' 'cat__month_4'
 'cat__month_5' 'cat__month_6' 'cat__month_7' 'cat__month_8'
 'cat__month_9' 'cat__month_10' 'cat__month_11' 'cat__month_12'
 'cat__year_2012' 'cat__year_2013' 'cat__year_2014' 'cat__year_2015'
 'cat__year_2016' 'cat__year_2017' 'cat__year_2018']


In [146]:
# Find the index of the 'price' column
quantity_index = list(feature_names).index('num__price')

# Create the monotone constraints list
monotone_constraints = [0] * len(feature_names)
monotone_constraints[quantity_index] = -1  # Enforce negative relationship for 'price'

# Define the pipeline with XGBRegressor and monotone constraints
pipeline_xgb_monotonic = Pipeline([
    ('date_transformer', date_transformer),
    ('preprocessor', preprocessor),
    ('regressor', XGBRegressor(monotone_constraints=monotone_constraints, random_state=3))
])

In [148]:
# Crear un diccionario de restricciones de monotonía donde la columna 'num__price' tiene un valor de -1 (relación inversa)
monotone_constraints_dict = {name: 0 for name in feature_names}  # Todos son 0
monotone_constraints_dict['num__price'] = -1  # Restringimos la columna 'num__price'

# Crear un nuevo modelo XGBRegressor con las restricciones de monotonía
xgb_monotonic = XGBRegressor(
    monotone_constraints=monotone_constraints_dict,
    random_state=42
)

# Reemplazar el regressor en el pipeline por el nuevo modelo con restricciones
pipeline_xgb_monotonic = Pipeline([
    ('date_transformer', date_transformer),
    ('preprocessor', preprocessor),
    ('regressor', xgb_monotonic)
])

# Entrenar el pipeline nuevamente
pipeline_xgb_monotonic.fit(X_train, y_train)

# Hacer predicciones y calcular el MAE
val_preds = pipeline_xgb_monotonic.predict(X_val)
mae_monotonic = mean_absolute_error(y_val, val_preds)
print(f'MAE con restricción de monotonia: {mae_monotonic}')

MAE con restricción de monotonia: 2524.0239778182536


In [149]:
# Guardar el modelo en un archivo .pkl
joblib.dump(pipeline_xgb_monotonic, 'pipeline_xgb_monotonic.pkl')

['pipeline_xgb_monotonic.pkl']

## 1.3 Optimización de Hiperparámetros con Optuna (20 puntos)

<p align="center">
  <img src="https://media.tenor.com/fmNdyGN4z5kAAAAi/hacking-lucy.gif">
</p>

Luego de presentarle sus resultados, Fiu le pregunta si es posible mejorar *aun más* su modelo. En particular, le comenta de la optimización de hiperparámetros con metodologías bayesianas a través del paquete `optuna`. Como usted es un aficionado al entrenamiento de modelos de ML, se propone implementar la descabellada idea de su jefe.

A partir de la mejor configuración obtenida en la sección anterior, utilice `optuna` para optimizar sus hiperparámetros. En particular, se pide que su optimización considere lo siguiente:

- Fijar una semilla en las instancias necesarias para garantizar la reproducibilidad de resultados
- Utilice `TPESampler` como método de muestreo
- De `XGBRegressor`, optimice los siguientes hiperparámetros:
    - `learning_rate` buscando valores flotantes en el rango (0.001, 0.1)
    - `n_estimators` buscando valores enteros en el rango (50, 1000)
    - `max_depth` buscando valores enteros en el rango (3, 10)
    - `max_leaves` buscando valores enteros en el rango (0, 100)
    - `min_child_weight` buscando valores enteros en el rango (1, 5)
    - `reg_alpha` buscando valores flotantes en el rango (0, 1)
    - `reg_lambda` buscando valores flotantes en el rango (0, 1)
- De `OneHotEncoder`, optimice el hiperparámetro `min_frequency` buscando el mejor valor flotante en el rango (0.0, 1.0)

Para ello se pide los siguientes pasos:
1. Implemente una función `objective()` que permita minimizar el `MAE` en el conjunto de validación. Use el método `.set_user_attr()` para almacenar el mejor pipeline entrenado. [10 puntos]
2. Fije el tiempo de entrenamiento a 5 minutos. [1 punto]
3. Optimizar el modelo y reportar el número de *trials*, el `MAE` y los mejores hiperparámetros encontrados. ¿Cómo cambian sus resultados con respecto a la sección anterior? ¿A qué se puede deber esto? [3 puntos]
4. Explique cada hiperparámetro y su rol en el modelo. ¿Hacen sentido los rangos de optimización indicados? [5 puntos]
5. Guardar su modelo en un archivo .pkl [1 punto]

In [150]:
import optuna
from optuna.samplers import TPESampler
optuna.logging.set_verbosity(optuna.logging.WARNING)

def objective(trial):
    # Hiperparámetros a optimizar
    learning_rate = trial.suggest_float('learning_rate', 0.001, 0.1)
    n_estimators = trial.suggest_int('n_estimators', 50, 1000)
    max_depth = trial.suggest_int('max_depth', 3, 10)
    max_leaves = trial.suggest_int('max_leaves', 0, 100)
    min_child_weight = trial.suggest_int('min_child_weight', 1, 5)
    reg_alpha = trial.suggest_float('reg_alpha', 0.0, 1.0)
    reg_lambda = trial.suggest_float('reg_lambda', 0.0, 1.0)
    min_frequency = trial.suggest_float('min_frequency', 0.0, 1.0)
    
    # Crear el modelo
    xgb_monotonic_opt = XGBRegressor(
        learning_rate=learning_rate,
        n_estimators=n_estimators,
        max_depth=max_depth,
        max_leaves=max_leaves,
        min_child_weight=min_child_weight,
        reg_alpha=reg_alpha,
        reg_lambda=reg_lambda,
        random_state=42,
        #monotone_constraints=monotone_constraints
    )
    encoder = OneHotEncoder(min_frequency=min_frequency, handle_unknown='ignore', sparse_output=False)
    
    preprocessor = ColumnTransformer(
        transformers=[
            ('num', StandardScaler(), numerical_cols),
            ('cat', encoder, categorical_cols)
        ]
    ).set_output(transform= 'pandas')
    pipeline = Pipeline(steps=[
        ('date_transformer', date_transformer),
        ('preprocessor', preprocessor),
        ('regressor', xgb_monotonic_opt)
    ])
    
    # Entrenar el pipeline
    pipeline.fit(X_train, y_train)
    
    # Predecir y calcular MAE en el conjunto de validación
    val_preds = pipeline.predict(X_val)
    mae = mean_absolute_error(y_val, val_preds)
    
    # Almacenar el mejor pipeline entrenado
    trial.set_user_attr('best_pipeline', pipeline)
    
    return mae

In [151]:
# Crear un estudio con TPESampler
study = optuna.create_study(direction='minimize', sampler=TPESampler(seed=42))

# Ejecutar la optimización con límite de tiempo de 5 minutos
study.optimize(objective, timeout=30)  # 300 segundos = 5 minutos

In [152]:
best_trial = study.best_trial
print(f"Number of trials: {len(study.trials)}")
print(f"Best MAE: {best_trial.value}")
print(f"Best hyperparameters: {best_trial.params}")

Number of trials: 37
Best MAE: 2005.0433978703375
Best hyperparameters: {'learning_rate': 0.08344666466543191, 'n_estimators': 577, 'max_depth': 9, 'max_leaves': 93, 'min_child_weight': 5, 'reg_alpha': 0.725569997204854, 'reg_lambda': 0.2432359796388749, 'min_frequency': 0.06336581659673984}


In [153]:
#save the best model in a .pkl file
joblib.dump(best_trial.user_attrs['best_pipeline'], 'pipeline_xgb_opt.pkl')

['pipeline_xgb_opt.pkl']

## 4. Optimización de Hiperparámetros con Optuna y Prunners (17 puntos)

<p align="center">
  <img src="https://i.pinimg.com/originals/90/16/f9/9016f919c2259f3d0e8fe465049638a7.gif">
</p>

Después de optimizar el rendimiento de su modelo varias veces, Fiu le pregunta si no es posible optimizar el entrenamiento del modelo en sí mismo. Después de leer un par de post de personas de dudosa reputación en la *deepweb*, usted llega a la conclusión que puede cumplir este objetivo mediante la implementación de **Prunning**.

Vuelva a optimizar los mismos hiperparámetros que la sección pasada, pero esta vez utilizando **Prunning** en la optimización. En particular, usted debe:

- Responder: ¿Qué es prunning? ¿De qué forma debería impactar en el entrenamiento? [2 puntos]
- Redefinir la función `objective()` utilizando `optuna.integration.XGBoostPruningCallback` como método de **Prunning** [10 puntos]
- Fijar nuevamente el tiempo de entrenamiento a 5 minutos [1 punto]
- Reportar el número de *trials*, el `MAE` y los mejores hiperparámetros encontrados. ¿Cómo cambian sus resultados con respecto a la sección anterior? ¿A qué se puede deber esto? [3 puntos]
- Guardar su modelo en un archivo .pkl [1 punto]

Nota: Si quieren silenciar los prints obtenidos en el prunning, pueden hacerlo mediante el siguiente comando:

```
optuna.logging.set_verbosity(optuna.logging.WARNING)
```

De implementar la opción anterior, pueden especificar `show_progress_bar = True` en el método `optimize` para *más sabor*.

Hint: Si quieren especificar parámetros del método .fit() del modelo a través del pipeline, pueden hacerlo por medio de la siguiente sintaxis: `pipeline.fit(stepmodelo__parametro = valor)`

Hint2: Este <a href = https://stackoverflow.com/questions/40329576/sklearn-pass-fit-parameters-to-xgboost-in-pipeline>enlace</a> les puede ser de ayuda en su implementación

**Qué es Prunning?** Pruning es una técnica que consiste en detener los trials que no están logrando un buen rendimiento durante las primeras etapas del entrenamiento. En lugar de continuar entrenando un modelo que probablemente no alcance el rendimiento deseado, el algoritmo de pruning finaliza su ejecución de forma anticipada, lo que permite reasignar los recursos a trials más prometedores.
El impacto de esta técnica es reducir el tiempo total de entrenamiento y optimización, ya que se evita gastar tiempo en configuraciones de hiperparámetros que claramente no proporcionarán buenos resultados. El pruning debería mejorar la eficiencia del proceso de optimización, permitiendo explorar más configuraciones de hiperparámetros en menos tiempo y enfocarse en las configuraciones con mayor potencial.

In [ ]:
#!pip install optuna-integration[xgboost]

In [105]:
import optuna
from optuna.integration import XGBoostPruningCallback
from optuna.samplers import TPESampler

def objective(trial):
    # Hiperparámetros a optimizar
    learning_rate = trial.suggest_float('learning_rate', 0.001, 0.1)
    n_estimators = trial.suggest_int('n_estimators', 50, 1000)
    max_depth = trial.suggest_int('max_depth', 3, 10)
    max_leaves = trial.suggest_int('max_leaves', 0, 100)
    min_child_weight = trial.suggest_int('min_child_weight', 1, 5)
    reg_alpha = trial.suggest_float('reg_alpha', 0.0, 1.0)
    reg_lambda = trial.suggest_float('reg_lambda', 0.0, 1.0)
    min_frequency = trial.suggest_float('min_frequency', 0.0, 1.0)
    
    # Crear el modelo
    xgb_monotonic_opt = XGBRegressor(
        learning_rate=learning_rate,
        n_estimators=n_estimators,
        max_depth=max_depth,
        max_leaves=max_leaves,
        min_child_weight=min_child_weight,
        reg_alpha=reg_alpha,
        reg_lambda=reg_lambda,
        random_state=42,
        #monotone_constraints=monotone_constraints,
        enable_categorical=True

    )
    encoder = OneHotEncoder(min_frequency=min_frequency, handle_unknown='ignore', sparse_output=False)
    
    preprocessor = ColumnTransformer(
        transformers=[
            ('num', StandardScaler(), numerical_cols),
            ('cat', encoder, categorical_cols)
        ]
    ).set_output(transform= 'pandas')
    pipeline = Pipeline([
        ('date_transformer', date_transformer),
        ('preprocessor', preprocessor),
        ('regressor', xgb_monotonic_opt)
    ])
    
    # Entrenar el pipeline con el callback de pruning
    pipeline.fit(
        X_train, y_train,
        regressor__eval_set=[(X_val, y_val)],
        regressor__early_stopping_rounds=10,
        regressor__callbacks=[XGBoostPruningCallback(trial, "validation_0-mae")]
    )
    
    # Predecir y calcular MAE en el conjunto de validación
    val_preds = pipeline.predict(X_val)
    mae = mean_absolute_error(y_val, val_preds)
    
    # Almacenar el mejor pipeline entrenado
    trial.set_user_attr('best_pipeline', pipeline)
    
    return mae

In [112]:
import optuna
from optuna.integration import XGBoostPruningCallback
from optuna.samplers import TPESampler

def objective(trial):
    # Hiperparámetros a optimizar
    learning_rate = trial.suggest_float('learning_rate', 0.001, 0.1)
    n_estimators = trial.suggest_int('n_estimators', 50, 1000)
    max_depth = trial.suggest_int('max_depth', 3, 10)
    max_leaves = trial.suggest_int('max_leaves', 0, 100)
    min_child_weight = trial.suggest_int('min_child_weight', 1, 5)
    reg_alpha = trial.suggest_float('reg_alpha', 0.0, 1.0)
    reg_lambda = trial.suggest_float('reg_lambda', 0.0, 1.0)
    min_frequency = trial.suggest_float('min_frequency', 0.0, 1.0)

    # Asegúrate de que X y y estén definidos antes de dividir
    X = df.drop(columns=['quantity'])  # Asegúrate de que 'quantity' es la columna objetivo
    y = df['quantity']


    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)
    # Transformación de las columnas de fecha
    X_train['year'] = X_train['date'].dt.year
    X_train['month'] = X_train['date'].dt.month
    X_train['day'] = X_train['date'].dt.day
    X_val['year'] = X_val['date'].dt.year
    X_val['month'] = X_val['date'].dt.month
    X_val['day'] = X_val['date'].dt.day

    # Eliminamos la columna 'date' que causa conflicto
    X_train = X_train.drop(columns=['date'])
    X_val = X_val.drop(columns=['date'])

    # Crear el modelo
    xgb_monotonic_opt = XGBRegressor(
        learning_rate=learning_rate,
        n_estimators=n_estimators,
        max_depth=max_depth,
        max_leaves=max_leaves,
        min_child_weight=min_child_weight,
        reg_alpha=reg_alpha,
        reg_lambda=reg_lambda,
        random_state=42,
        enable_categorical=True  # Asegúrate de que esto esté habilitado
    )

    # Codificar categorías y escalar variables numéricas
    encoder = OneHotEncoder(min_frequency=min_frequency, handle_unknown='ignore', sparse_output=False)

    preprocessor = ColumnTransformer(
        transformers=[
            ('num', StandardScaler(), numerical_cols),
            ('cat', encoder, categorical_cols)
        ]
    ).set_output(transform='pandas')

    pipeline = Pipeline([
        ('date_transformer', date_transformer),
        ('preprocessor', preprocessor),
        ('regressor', xgb_monotonic_opt)
    ])

    # Entrenar el pipeline con el callback de pruning
    pipeline.fit(
        X_train, y_train,
        regressor__eval_set=[(X_val, y_val)],
        regressor__early_stopping_rounds=10,
        regressor__callbacks=[XGBoostPruningCallback(trial, "validation_0-mae")]
    )

    # Predecir y calcular MAE en el conjunto de validación
    val_preds = pipeline.predict(X_val)
    mae = mean_absolute_error(y_val, val_preds)

    # Almacenar el mejor pipeline entrenado
    trial.set_user_attr('best_pipeline', pipeline)

    return mae


In [113]:
# Crear un estudio con TPESampler
study = optuna.create_study(direction='minimize', sampler=TPESampler(seed=42))

# Ejecutar la optimización con límite de tiempo de 5 minutos
study.optimize(objective, timeout=30)  # 300 segundos = 5 minutos

[W 2024-10-23 00:01:29,521] Trial 0 failed with parameters: {'learning_rate': 0.03807947176588889, 'n_estimators': 954, 'max_depth': 8, 'max_leaves': 60, 'min_child_weight': 1, 'reg_alpha': 0.15599452033620265, 'reg_lambda': 0.05808361216819946, 'min_frequency': 0.8661761457749352} because of the following error: KeyError('date').
Traceback (most recent call last):
  File "c:\Users\nicol\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas\core\indexes\base.py", line 3802, in get_loc
    return self._engine.get_loc(casted_key)
  File "pandas\_libs\index.pyx", line 138, in pandas._libs.index.IndexEngine.get_loc
  File "pandas\_libs\index.pyx", line 165, in pandas._libs.index.IndexEngine.get_loc
  File "pandas\_libs\hashtable_class_helper.pxi", line 5745, in pandas._libs.hashtable.PyObjectHashTable.get_item
  File "pandas\_libs\hashtable_class_helper.pxi", line 5753, in pandas._libs.hashtable.PyObjectHashTable.get_item
KeyError: 'date'

The above exception was the direct caus

KeyError: 'date'

## 5. Visualizaciones (5 puntos)

<p align="center">
  <img src="https://media.tenor.com/F-LgB1xTebEAAAAd/look-at-this-graph-nickelback.gif">
</p>


Satisfecho con su trabajo, Fiu le pregunta si es posible generar visualizaciones que permitan entender el entrenamiento de su modelo.

A partir del siguiente <a href = https://optuna.readthedocs.io/en/stable/tutorial/10_key_features/005_visualization.html#visualization>enlace</a>, genere las siguientes visualizaciones:

1. Gráfico de historial de optimización [1 punto]
2. Gráfico de coordenadas paralelas [1 punto]
3. Gráfico de importancia de hiperparámetros [1 punto]

Comente sus resultados:

4. ¿Desde qué *trial* se empiezan a observar mejoras notables en sus resultados? [0.5 puntos]
5. ¿Qué tendencias puede observar a partir del gráfico de coordenadas paralelas? [1 punto]
6. ¿Cuáles son los hiperparámetros con mayor importancia para la optimización de su modelo? [0.5 puntos]

In [ ]:
# Inserte su código acá

## 6. Síntesis de resultados (3 puntos)

Finalmente:

1. Genere una tabla resumen del MAE en el conjunto de validación obtenido en los 5 modelos entrenados desde Baseline hasta XGBoost con Constraints, Optuna y Prunning. [1 punto]
2. Compare los resultados de la tabla y responda, ¿qué modelo obtiene el mejor rendimiento? [0.5 puntos]
3. Cargue el mejor modelo, prediga sobre el conjunto de **test** y reporte su MAE. [0.5 puntos]
4. ¿Existen diferencias con respecto a las métricas obtenidas en el conjunto de validación? ¿Porqué puede ocurrir esto? [1 punto]

In [ ]:
# Inserte su código acá

# Conclusión
Eso ha sido todo para el lab de hoy, recuerden que el laboratorio tiene un plazo de entrega de una semana. Cualquier duda del laboratorio, no duden en contactarnos por mail o U-cursos.

<p align="center">
  <img src="https://media.tenor.com/8CT1AXElF_cAAAAC/gojo-satoru.gif">
</p>

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=87110296-876e-426f-b91d-aaf681223468' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>